In [1]:
from mybatch import *
from data import *
import numpy as np

from dataset import Dataset, DatasetIndex

## Linear regression allows solving the task where the answer is a continuous variable.
For example: If you needed to predict the price of a house - target value. Your features - the area in square meters, the number of  windows, the number of rooms and etc. The response variable is the price of this house.

Our model tries to predict this price. After the training the model will give its answer, and we need to understand how well the model predicts. 

If the price of house is 25000 and model predicts 23300, we can calculate the percentage of model error. In this case percentage been
### $ 100 - \frac{23300 * 100}{25000} = 6.79\% $
And if we predict a lot of prices, we can calculate the average percentage of errors.

Let's train the Linear regerssion and calculate the error.

In [22]:
size = 1000
linear_x, linear_y = generate_linear_data(size=size, error_scale=1)

In [23]:
dataset = Dataset(np.arange(size), batch_class=MyBatch, preloaded=(linear_x, linear_y))
dataset.cv_split()

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [25]:
dict_params = {'loss': [], 'w': [], 'b': []}

dataset.train.p\
    .load((linear_x, linear_y))\
    .train_linear_model(sess, dict_params)\
    .run(10, n_epochs=3)

In [26]:
predict = []

dataset.test.p\
    .load((linear_x, linear_y))\
    .predict_linear(sess, predict=predict)\
    .run(int(size * 0.2),n_epochs=1)

In [27]:
predict = np.array(predict).reshape(-1)
target = np.array(linear_y[dataset.test.indices]).reshape(-1)

percent = np.mean(np.abs((target - predict) * 100 / target))

print('percentage of errors: {}%'.format(round(percent, 3)))

percentage of errors: 3.788%


# Logistic regression solve the task in the case where answer is binary (0 or 1, -1 or 1 and etc)
For example:
You need to predict the sex of human by weight and height. 

Our model can give a persentage of belonging to one of the classes, and you can choose a threshold after which
the prediction belongs to class man or class woman.

For understand how well our models work, we can predict some data and calculate the percentage of true values.

Let's train model and calculate percentage of true values.

In [8]:
size = 1000
logistic_x, logistic_y = generate_logistic_data(size, [[1,2],[[15,0],[0,15]]], [[10,17],[[15,0],[0,15]]])

In [9]:
dataset = Dataset(np.arange(size), batch_class=MyBatch, preloaded=(logistic_x, logistic_y))
dataset.cv_split()

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
dict_params = {'loss': [], 'w': [], 'b': []}

dataset.train.p\
    .load((logistic_x, logistic_y))\
    .train_logistic_model(sess, dict_params)\
    .run(10, n_epochs=10)

In [12]:
predict = []

dataset.test.p\
    .load((logistic_x, logistic_y))\
    .predict_logistic(sess, predict=predict)\
    .run(int(size * 0.2),n_epochs=1)

In [13]:
predict = np.array([0 if i <0.5 else 1 for i in np.array(predict).reshape(-1)])
target = np.array(logistic_y[dataset.test.indices].reshape(-1), dtype=np.int32)

percentage = np.sum(np.where(target == predict)) / len(target)
print('Percentage of true values: {}%'.format(round(percentage, 1)))

Percentage of true values: 98.1%


# poisson regression is used if the answer is count. 
For example: your task is to count the number of people in a restaurant by the time of day and time of year.

But these are not permanent values, because yesterday at 14:00 there may be 5 people, and today at 14:00 there may be 6 people. And if our model predicts that tomorrow 5 people will come, but 6 come, we can not say that our model is bad, because the probability that 5 or 6 people comes very close.

Therefose, we can calculate variance of true values, which show how much the number of people fluctuates at this time. After train the model, we can calculate variance of the prediction value, and if true variance be eual predict variance it means what out model is works normally.

Let's fit model and percantage of errors and variance ratio.

In [14]:
size = 1000
poisson_x, poisson_y = generate_poisson_data(np.random.random(13), size)

In [15]:
dataset = Dataset(np.arange(size), batch_class=MyBatch, preloaded=(poisson_x, poisson_y))
dataset.cv_split()

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
dict_params = {'loss': [], 'w': [], 'b': []}

dataset.train.p\
    .load((poisson_x, poisson_y))\
    .train_poisson_model(sess, dict_params)\
    .run(100, n_epochs=150)

In [18]:
predict = []

dataset.test.p\
    .load((poisson_x, poisson_y))\
    .predict_poisson(sess, predict=predict)\
    .run(int(size * 0.2),n_epochs=1)

In [19]:
predict = np.array(predict).reshape(-1)
predict = np.array([round(i) for i in predict])
target = np.array(poisson_y[dataset.test.indices]).reshape(-1)

true_var = np.mean((target - np.mean(target))**2)
predict_var = np.mean((predict - np.mean(predict))**2)

percent = 
print('Percentage of errors: {}%'.format(round(percent, 3)), 'Variance raito: %.3f' % (predict_var / true_var))

Percentage of errors: 11.025% Variance raito: 0.919
